In [5]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

In [6]:
clean_data_dir = Path('../data/clean')

apps_to_turkey = pd.read_csv(clean_data_dir / 'applications_to_turkey_clean.csv')
apps_from_turkey = pd.read_csv(clean_data_dir / 'applications_from_turkey_clean.csv')
decisions_turkey = pd.read_csv(clean_data_dir / 'decisions_from_turkey_clean.csv')
decisions_from_others = pd.read_csv(clean_data_dir / 'decisions_from_other_countries_clean.csv')

print('Data loaded successfully!')

visualizations_dir = Path('../visualizations')
visualizations_dir.mkdir(exist_ok=True)
print(f"Visualizations will be saved to: {visualizations_dir}")

Data loaded successfully!
Visualizations will be saved to: ..\visualizations


# Interactive Visualizations

## Goals:
1. **Interactive Choropleth Map** - Color countries by number of applications/recognition rate
2. **Animated Time Series Map** - Show how refugee flows changed year by year
3. **Flow Map** - Arrows from origin countries to Turkey (if feasible)
4. **Interactive Dashboard** - Filterable charts with dropdowns for year, country, etc.

We'll start with the choropleth map and build from there.

In [7]:
apps_by_country = apps_to_turkey.groupby(['Country of Origin Name', 'Country of Origin Code', 'Year'])['Number of Applications'].sum().reset_index()

recognition_data = decisions_turkey.groupby(['Country of Origin Name', 'Year']).agg({
    'Recognized': 'sum',
    'Total Decided': 'sum'
}).reset_index()

recognition_data['Recognition Rate'] = (
    recognition_data['Recognized'] / recognition_data['Total Decided'] * 100
).round(1)


apps_by_country = apps_by_country.merge(
    recognition_data[['Country of Origin Name', 'Year', 'Recognition Rate']], 
    on=['Country of Origin Name', 'Year'], 
    how='left'  # Left join so we keep all applications even if no decision data
)

apps_by_country['Recognition Rate'] = apps_by_country['Recognition Rate'].apply(
    lambda x: f"{x:.1f}%" if pd.notna(x) else 'No data'
)

apps_by_country.columns = ['Country', 'ISO_Code', 'Year', 'Applications', 'Recognition Rate']

# Cut off at 2018 because of the Temporary Protection status of migrants after 2018 which caused data incompleteness.
apps_by_country_filtered = apps_by_country[apps_by_country['Year'] <= 2018]

print(apps_by_country.sample(10))

                  Country ISO_Code  Year  Applications Recognition Rate
427               Nigeria      NGA  2010            20             0.0%
22            Afghanistan      AFG  2022         19400            19.3%
650  United Arab Emirates      ARE  2014             7          No data
236                 India      IND  2017             5          No data
383              Mongolia      MNG  2017            34             0.0%
684                 Yemen      YEM  2013           215            28.8%
523             Sri Lanka      LKA  2013             9          No data
588  Syrian Arab Republic      SYR  2013           123            54.5%
624          Turkmenistan      TKM  2005             8           100.0%
359                 Libya      LBY  2016            23            56.5%


In [8]:
fig = px.choropleth(
    apps_by_country_filtered,
    locations='ISO_Code',                 
    color='Applications',              
    hover_name='Country',
    animation_frame='Year',
    hover_data={'ISO_Code': False,
                'Year': False,
                'Applications': ':,',
                'Recognition Rate': True
                },
    color_continuous_scale='Viridis',       
    labels={'Applications': 'Applications',
            'Recognition Rate': 'Recognition Rate'},
    title='Total Asylum Applications to Turkey by Origin Country (2000-2018)',
)

fig.update_geos(
    fitbounds='locations',
    visible=False,
    showcountries=True
)

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    ),
    height=600
)

fig.update_coloraxes(
    colorbar=dict(
        title="Number of<br>Applications",
        thickness=15,
        len=0.7
    )
)

fig.add_trace(
    go.Choropleth(
        locations=['TUR'],
        z=[1],
        colorscale=[[0, 'red'], [1, 'red']],
        showscale=False,
        hoverinfo='skip',
        name='Turkey'
    )
)

fig.add_annotation(
    text="Source: UNHCR Population Statistics Database",
    xref="paper", yref="paper",
    x=0, y=-0.1,
    showarrow=False,
    font=dict(size=10, color="gray")
)

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1000
fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 500

fig.show()

fig.write_html(visualizations_dir / "turkey_asylum_applications_map.html")